In [1]:
import matplotlib.pyplot as plt
import pandas as pd 
import spotipy 
import librosa
import librosa.display
import numpy as np
import matplotlib as mpl
import urllib.request
import seaborn as sns
import tqdm
import scipy
import joblib
from pathlib import Path
from IPython.display import Audio, Markdown, Image
from spotipy.oauth2 import SpotifyClientCredentials


# Números y Datos
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from scipy.interpolate import interp1d

# Análisis de sonido
import librosa
import librosa.display
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials 

from scipy.signal import savgol_filter

# Machine learning
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

sns.set_context('poster')
sns.set_style('darkgrid')

plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

In [4]:
!wget https://www.dropbox.com/s/ms6260jqmsmz5qu/audio_features.pickle?dl=1 -O audio_features.pickle
!wget https://www.dropbox.com/s/8d0y9bms80ly8ui/tracks.pickle?dl=1 -O tracks.pickle
!wget https://www.dropbox.com/s/1cmb8e8zvv6hsfs/audio_analysis.pickle?dl=1 -O audio_analysis.pickle

'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.


## Levantar el dataset AA

In [3]:
#df_af = pd.read_pickle('audio_features.pickle')
#df_tracks = pd.read_pickle('tracks.pickle')
df_aa = pd.read_pickle('./features/audio_analysis.pickle') \
    .set_index(['id','start']) \
    .sort_values(by=['id', 'start'], ascending=True, na_position='first')

FileNotFoundError: [Errno 2] No such file or directory: './features/audio_analysis.pickle'

## Desglozar las listas de pitches y timbre

In [4]:
df_m2 = df_aa

pitches_nombre = ['p00_C','p01_C#','p02_D','p03_D#','p04_E','p05_F',
                 'p06_F#','p07_G','p08_G#','p09_A','p10_A#','p11_B']
df1 = pd.DataFrame(df_m2['pitches'].tolist(), columns=pitches_nombre, index=df_m2.index)
df_m2 = pd.concat([df_m2,df1],axis=1)

timbre_nombre = ['t00','t01','t02','t03','t04','t05',
                 't06','t07','t08','t09','t10','t11']
df1 = pd.DataFrame(df_m2['timbre'].tolist(), columns=timbre_nombre, index=df_m2.index)
df_m2 = pd.concat([df_m2,df1],axis=1)

df_m2 = df_m2.drop(['pitches','timbre'],1)

df1 = None
df_m2.head(3).T

id                00At7PWydsvg7g5xgaYan9                      
start                            0.00000    0.24381    0.41873
duration                         0.24381    0.17492    0.17329
confidence                       0.00000    1.00000    1.00000
loudness_start                 -60.00000  -60.00000  -34.27900
loudness_max_time                0.00000    0.01436    0.01554
loudness_max                   -60.00000  -12.45700  -12.72800
loudness_end                     0.00000    0.00000    0.00000
p00_C                            1.00000    0.12100    0.03200
p01_C#                           1.00000    0.99600    1.00000
p02_D                            1.00000    0.08000    0.06000
p03_D#                           1.00000    0.10400    0.05900
p04_E                            1.00000    1.00000    0.36800
p05_F                            1.00000    0.22300    0.08400
p06_F#                           1.00000    0.04300    0.03600
p07_G                            1.00000    0.04700    0.04100
p08_G#                           1.00000    0.17600    0.10600
p09_A                            1.00000    0.42200    0.38000
p10_A#                           1.00000    0.12900    0.11200
p11_B                            1.00000    0.06200    0.08700
t00                              0.00000   37.64100   38.17100
t01                            171.13000  -77.64300  -17.74100
t02                              9.46900  -30.61300  -22.79800
t03                            -28.48000  137.46200  149.22300
t04                             57.49100   36.96000   31.83300
t05                            -50.06700  -51.02400  -53.44600
t06                             14.83300  -18.74500  -36.48500
t07                              5.35900  -28.82200  -39.47900
t08                            -27.22800   -7.17000  -25.79500
t09                              0.97300   -1.99400   -1.90300
t10                            -10.64000  -15.69500  -21.90800
t11                             -7.22800    9.41100   15.93400

## Obtener los cuartiles para sintetizar la info más importante

In [5]:
df_m2_25 = df_m2.groupby('id').quantile(.25)
df_m2_50 = df_m2.groupby('id').quantile(.5)
df_m2_75 = df_m2.groupby('id').quantile(.75)

In [6]:
df_m2_75.head(3).T

id,00At7PWydsvg7g5xgaYan9,00BgcUYkIImsDN5rNjkR1U,00WgnFIsY1y84p7mYAJRuF
duration,0.272995,0.33900,0.269863
confidence,0.695000,0.71100,0.702000
loudness_start,-5.628500,-5.74900,-11.602000
loudness_max_time,0.057285,0.03646,0.035515
loudness_max,-0.174000,-0.42700,-3.615750
loudness_end,0.000000,0.00000,0.000000
p00_C,0.507500,0.39100,0.884250
p01_C#,1.000000,1.00000,0.585250
p02_D,0.405000,0.20600,0.631000
p03_D#,0.378000,0.19800,0.915250


In [7]:
df_m2_25.to_pickle("./audio_analysis_25.pickle")
df_m2_50.to_pickle("./audio_analysis_50.pickle")
df_m2_75.to_pickle("./audio_analysis_75.pickle")

## Levantar las otras fuentes de datos

In [9]:
df_af = pd.read_pickle('../features/audio_features.pickle')
df_tracks = pd.read_pickle('../features/tracks.pickle')
df_af = df_af.drop(['duration_ms','type','uri','track_href','analysis_url'],1)
df_tracks = df_tracks.drop(['uri','type','preview_url','href','external_urls','external_ids'],1)
df_tracks.head(3)


,album,artists,available_markets,disc_number,duration_ms,explicit,is_local,name,popularity,track_number,genre
id,,,,,,,,,,,
4d3XHYFFuYYzxWr2cJ6yQl,"{'album_type': 'ALBUM', 'artists': [{'external...",Tommy Flanagan,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",1,349600,False,False,Angel Eyes - Studio,49,3,jazz
3X3p3u03P8eFL8WTH0oaaU,"{'album_type': 'ALBUM', 'artists': [{'external...",Art Farmer,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",1,344066,False,False,Fair Weather - Remastered,47,2,jazz
4KlXeaheot0OI9PoOWspvZ,"{'album_type': 'ALBUM', 'artists': [{'external...",Christian Scott aTunde Adjuah,"[CA, US]",1,348746,False,False,Cease Fire,13,5,jazz


## Mergear los datasets 
(inner join para maximizar la certeza de nuestro análisis)

In [11]:
df_m = df_tracks.merge(df_af, on='id', how='inner')
df_m = df_m.merge(df_m2_25.add_suffix('_25'), on='id', how='inner')
df_m = df_m.merge(df_m2_50.add_suffix('_50'), on='id', how='inner')
df_m = df_m.merge(df_m2_75.add_suffix('_75'), on='id', how='inner')
df_m.head(3)

,album,artists,available_markets,disc_number,duration_ms,explicit,is_local,name,popularity,track_number,...,t02_75,t03_75,t04_75,t05_75,t06_75,t07_75,t08_75,t09_75,t10_75,t11_75
id,,,,,,,,,,,,,,,,,,,,,
4d3XHYFFuYYzxWr2cJ6yQl,"{'album_type': 'ALBUM', 'artists': [{'external...",Tommy Flanagan,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",1,349600,False,False,Angel Eyes - Studio,49,3,...,45.6380,69.10000,46.32300,-11.0760,26.7100,12.6140,8.86100,15.50700,-1.12600,13.90700
3X3p3u03P8eFL8WTH0oaaU,"{'album_type': 'ALBUM', 'artists': [{'external...",Art Farmer,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",1,344066,False,False,Fair Weather - Remastered,47,2,...,42.6055,27.08675,46.55975,-6.9520,17.8815,21.0810,-1.19175,10.23325,-1.91975,11.76825
4KlXeaheot0OI9PoOWspvZ,"{'album_type': 'ALBUM', 'artists': [{'external...",Christian Scott aTunde Adjuah,"[CA, US]",1,348746,False,False,Cease Fire,13,5,...,30.2320,61.02800,47.58400,-6.0845,10.7075,10.6685,14.46150,8.38950,2.20300,14.25750


In [12]:
df_m.to_pickle("./dataset_integrado.pickle")

In [14]:
df_m.genre

id
4d3XHYFFuYYzxWr2cJ6yQl      jazz
3X3p3u03P8eFL8WTH0oaaU      jazz
4KlXeaheot0OI9PoOWspvZ      jazz
3j76McoUI18gDoqxYE14Bu      jazz
4zxd4tiXPlWMqoJltbVTbE      jazz
                           ...  
5vSsQH8wH68PDKJbNfxzpc    trance
7xYKAlU77jAAnAixQ5IgSn    trance
4hXEgyy5nNgdiBvJcJ9o8u    trance
4HpSbJgNpoOphIeCK3z9sk    trance
30KzbX1viF0aTL1NS5coDY    trance
Name: genre, Length: 4082, dtype: object